In [1]:
import pandas as pd
import re
from collections import Counter

# Read the CSV
df = pd.read_csv('Digital_Digital_Access__Behavior_Survey_DAB_-_all_versions_-_English_eng_-_2025-12-29-02-53-07.csv', delimiter=';')

print(f"Raw CSV loaded: {df.shape}")

# FIRST: Handle duplicate columns from raw CSV (before any renaming)
# Pandas adds .1, .2, .3 suffixes when reading duplicate column names
import re as regex_lib
duplicate_cols_in_raw = {}

for i, col in enumerate(df.columns):
    # Check if this column has a .1, .2, etc suffix from pandas
    if regex_lib.search(r'\.\d+$', col):
        base_name = regex_lib.sub(r'\.\d+$', '', col)
        if base_name not in duplicate_cols_in_raw:
            duplicate_cols_in_raw[base_name] = []
        duplicate_cols_in_raw[base_name].append((i, col))

if duplicate_cols_in_raw:
    print(f"\nFound {len(duplicate_cols_in_raw)} sets of duplicate columns in raw CSV:")
    
    for base_name, cols in duplicate_cols_in_raw.items():
        print(f"\n  '{base_name[:80]}'...")
        
        # Find the base column too
        base_indices = [i for i, c in enumerate(df.columns) if c == base_name]
        all_cols = [(i, base_name) for i in base_indices] + cols
        
        # Find which column has the most 1s (actual data), not just non-null
        best_idx = all_cols[0][0]
        best_ones = 0
        best_non_null = 0
        
        for idx, col_name in all_cols:
            non_null = df.iloc[:, idx].notna().sum()
            ones = (df.iloc[:, idx] == 1.0).sum() + (df.iloc[:, idx] == 1).sum()
            
            print(f"    Column at index {idx} ('{col_name}'): {non_null} non-null, {ones} ones")
            
            # Prioritize columns with 1s, then by non-null count
            if ones > best_ones or (ones == best_ones and non_null > best_non_null):
                best_idx = idx
                best_ones = ones
                best_non_null = non_null
        
        print(f"    → Keeping column at index {best_idx} ({best_ones} ones, {best_non_null} non-null)")
        
        # Keep the best column's data in the base column name
        if best_idx != base_indices[0] if base_indices else -1:
            # Copy data from best column to base column
            if base_indices:
                df[base_name] = df.iloc[:, best_idx]
            else:
                # Base column doesn't exist, rename the best one
                df = df.rename(columns={df.columns[best_idx]: base_name})
        
        # Drop all suffix columns
        for idx, col_name in cols:
            if col_name in df.columns:
                df = df.drop(columns=[col_name])

print(f"\nAfter removing raw CSV duplicates: {df.shape}")

# NOW proceed with renaming
rename_map = {}
option_counters = {}

for col in df.columns:
    # Special case: Q01.1 -> Q00
    if col.startswith('Q01.1'):
        rename_map[col] = 'Q00'
        continue
    
    # Sub-question pattern: Q##[_-]text/option
    if match := re.match(r'(Q\d+)[_-](.*[?\)"])/(.*)', col):
        q_code = match.group(1)

        # Generate sub-code (Q10_a, Q10_b, etc.)
        if q_code not in option_counters:
            option_counters[q_code] = 0
        sub_code = f"{q_code}_{chr(97 + option_counters[q_code])}"
        option_counters[q_code] += 1

        rename_map[col] = sub_code

    # Follow-up pattern: Q##[_-]If other...
    elif match := re.match(r'(Q\d+)[_-](If .*)', col):
        q_code = match.group(1)
        follow_up_code = f"{q_code}_other_text"
        rename_map[col] = follow_up_code

    # Main question pattern: Q##[_-]text
    elif match := re.match(r'(Q\d+)[_-](.*)', col):
        q_code = match.group(1)
        rename_map[col] = q_code

# Create cleaned version with codes
df_cleaned = df.rename(columns=rename_map)

# Handle any duplicate column names created by renaming (shouldn't happen now, but keep as safety)
col_counts = Counter(df_cleaned.columns)
dup_names = {k: v for k, v in col_counts.items() if v > 1}

if dup_names:
    print(f"\nFound {len(dup_names)} duplicate column names after renaming")
    for dup_name in dup_names.keys():
        print(f"  Processing '{dup_name}': {dup_names[dup_name]} columns")
        
        dup_indices = [i for i, col in enumerate(df_cleaned.columns) if col == dup_name]
        
        # Find best column (most 1s, then most non-null)
        best_idx = dup_indices[0]
        best_ones = 0
        best_non_null = 0
        
        for idx in dup_indices:
            non_null = df_cleaned.iloc[:, idx].notna().sum()
            ones = (df_cleaned.iloc[:, idx] == 1.0).sum() + (df_cleaned.iloc[:, idx] == 1).sum()
            
            print(f"    Column at index {idx}: {non_null} non-null, {ones} ones")
            
            if ones > best_ones or (ones == best_ones and non_null > best_non_null):
                best_idx = idx
                best_ones = ones
                best_non_null = non_null
        
        print(f"    → Keeping column at index {best_idx} ({best_ones} ones, {best_non_null} non-null)")

        # Keep best column
        merged_col = df_cleaned.iloc[:, best_idx].copy()
        first_pos = min(dup_indices)

        # Drop all duplicates
        cols_to_drop = [df_cleaned.columns[i] for i in dup_indices]
        df_cleaned = df_cleaned.drop(columns=cols_to_drop)

        # Add back at first position
        cols_list = df_cleaned.columns.tolist()
        cols_list.insert(first_pos, dup_name)
        df_cleaned[dup_name] = merged_col
        df_cleaned = df_cleaned[cols_list]

print(f"After duplicate merging: {len(df_cleaned.columns)} columns, {df_cleaned.columns.nunique()} unique")

# Step 1: Consolidate Township columns
township_cols = [col for col in df_cleaned.columns if '(Township)' in col]
print(f"\nFound {len(township_cols)} Township columns")

def get_township(row):
    for col in township_cols:
        val = row[col]
        if pd.notna(val) and str(val).strip():
            return str(val).strip()
    return None

if township_cols:
    df_cleaned['Township'] = df_cleaned.apply(get_township, axis=1)
    print(f"Created Township column with {df_cleaned['Township'].notna().sum()} non-null values")

    region_idx = df_cleaned.columns.tolist().index('Region')
    cols = df_cleaned.columns.tolist()
    cols.remove('Township')
    cols.insert(region_idx + 1, 'Township')
    df_cleaned = df_cleaned[cols]
    print(f"Positioned Township column after Region column")

    df_cleaned = df_cleaned.drop(columns=township_cols)
    print(f"Dropped {len(township_cols)} individual township columns")

# Step 2: Convert 0/1 to Boolean
bool_conversion_count = 0
for col in df_cleaned.columns:
    try:
        unique_vals = df_cleaned[col].dropna().unique()
        if len(unique_vals) > 0:
            unique_str_vals = set(str(v) for v in unique_vals)
            if unique_str_vals.issubset({'0', '1', '0.0', '1.0'}):
                df_cleaned[col] = df_cleaned[col].map({
                    0: False, 1: True,
                    0.0: False, 1.0: True,
                    '0': False, '1': True
                })
                bool_conversion_count += 1
    except:
        pass

print(f"\nConverted {bool_conversion_count} binary columns to Boolean values")

# Step 3: Remove metadata columns
cols_to_remove = ['_id', '_uuid', '_submission_time', '_validation_status', '_notes', '_status',
                  '_submitted_by', '_tags', '_index', 'meta/instanceID', 'start', 'end',
                  '__version__', 'deviceid', 'subscriberid', 'simid', 'phonenumber']

existing_cols_to_remove = [col for col in cols_to_remove if col in df_cleaned.columns]
if existing_cols_to_remove:
    df_cleaned = df_cleaned.drop(columns=existing_cols_to_remove)
    print(f"Removed {len(existing_cols_to_remove)} metadata columns")

# Save output file
df_cleaned.to_csv('CLEAN_FY26.csv', index=False)

print(f"\nCreated CLEAN_FY26.csv ({len(df_cleaned)} rows, {len(df_cleaned.columns)} columns)")

Raw CSV loaded: (558, 278)

Found 4 sets of duplicate columns in raw CSV:

  'Q33_Which of the following Yetagon products and services have you used ? (Please'...
    Column at index 183 ('Q33_Which of the following Yetagon products and services have you used ? (Please select all that apply.)/Yetagon Tele-agronomy'): 529 non-null, 0 ones
    Column at index 184 ('Q33_Which of the following Yetagon products and services have you used ? (Please select all that apply.)/Yetagon Tele-agronomy.1'): 529 non-null, 264 ones
    → Keeping column at index 184 (264 ones, 529 non-null)

  'Direct or Digital?'...
    Column at index 3 ('Direct or Digital?'): 558 non-null, 0 ones
    Column at index 264 ('Direct or Digital?.1'): 0 non-null, 0 ones
    → Keeping column at index 3 (0 ones, 558 non-null)

  'Q26_[Confident] [other]  How do they use the Facebook? OR how confident are they'...
    Column at index 145 ('Q26_[Confident] [other]  How do they use the Facebook? OR how confident are they using 

# Generate CLEAN_FY26.csv from Digital Survey Data

This notebook processes the Digital Access & Behavior Survey raw data and creates a cleaned dataset for the dashboard.

## Important Notes on 'None of the above' Options

### Questions WITH 'None of the above':
- **Q31** (Yetagon products heard) → **Q31_j** = "None of the above"
- **Q33** (Yetagon products used) → **Q33_g** = "None of the above"

### Questions WITHOUT 'None of the above':
- **Q36** (Bio-product brands heard apart from Yetagon) - No "None of the above" option in survey
- **Q37** (Bio-product brands used apart from Yetagon) - No "None of the above" option in survey

The dashboard should display Q31_j and Q33_g properly. For Q36/Q37, the absence is intentional in the survey design.

In [2]:
# Verify key columns exist
print("\n=== Verification ===")

# Check Township column
if 'Township' in df_cleaned.columns:
    print(f"\n✓ Township column created successfully")
    print(f"  Total townships: {df_cleaned['Township'].notna().sum()} non-null values")
    print(f"  Unique townships: {df_cleaned['Township'].nunique()}")
    
    # Check position relative to Region
    region_idx = df_cleaned.columns.tolist().index('Region')
    township_idx = df_cleaned.columns.tolist().index('Township')
    if township_idx == region_idx + 1:
        print(f"  ✓ Township positioned correctly after Region (positions {region_idx}, {township_idx})")
    
    # Show sample values
    print(f"  Sample townships: {df_cleaned['Township'].dropna().head(10).tolist()}")
    
    # Show township distribution (top 10)
    print(f"\n  Top 10 townships by count:")
    print(df_cleaned['Township'].value_counts().head(10))
else:
    print("\n✗ WARNING: Township column not found!")

# Verify individual township columns were removed
township_cols_remaining = [col for col in df_cleaned.columns if '(Township)' in col]
if len(township_cols_remaining) == 0:
    print(f"\n✓ Individual township columns successfully removed")
else:
    print(f"\n✗ WARNING: {len(township_cols_remaining)} township columns still present: {township_cols_remaining}")

# Check Q31 (should include Q31_j - None of the above)
q31_cols = [col for col in df_cleaned.columns if col.startswith('Q31_')]
print(f"\n✓ Q31 columns ({len(q31_cols)}): {sorted(q31_cols)}")
print(f"  Q31_j (None of the above) present: {'Q31_j' in q31_cols}")

# Check Q33 (should include Q33_g - None of the above)
q33_cols = [col for col in df_cleaned.columns if col.startswith('Q33_')]
print(f"\n✓ Q33 columns ({len(q33_cols)}): {sorted(q33_cols)}")
print(f"  Q33_g (None of the above) present: {'Q33_g' in q33_cols}")

# Check Q36 (bio-product brands heard)
q36_cols = [col for col in df_cleaned.columns if col.startswith('Q36_')]
print(f"\n✓ Q36 columns ({len(q36_cols)}): {sorted(q36_cols)}")
print(f"  NOTE: Q36 does NOT have a 'None of the above' option in the original survey")

# Check Q37 (bio-product brands used)
q37_cols = [col for col in df_cleaned.columns if col.startswith('Q37_')]
print(f"\n✓ Q37 columns ({len(q37_cols)}): {sorted(q37_cols)}")
print(f"  NOTE: Q37 does NOT have a 'None of the above' option in the original survey")

# Sample statistics with Boolean values
if 'Q31_j' in df_cleaned.columns:
    q31_j_count = (df_cleaned['Q31_j'] == True).sum()
    q31_j_pct = (q31_j_count / len(df_cleaned) * 100) if len(df_cleaned) > 0 else 0
    print(f"\nQ31_j responses: {q31_j_count} ({q31_j_pct:.1f}%) selected 'None of the above'")
    print(f"Q31_j data type: {df_cleaned['Q31_j'].dtype}")

if 'Q33_g' in df_cleaned.columns:
    q33_g_count = (df_cleaned['Q33_g'] == True).sum()
    q33_g_pct = (q33_g_count / len(df_cleaned) * 100) if len(df_cleaned) > 0 else 0
    print(f"Q33_g responses: {q33_g_count} ({q33_g_pct:.1f}%) selected 'None of the above'")
    print(f"Q33_g data type: {df_cleaned['Q33_g'].dtype}")

# Verify Boolean conversion
print("\n=== Boolean Conversion Check ===")
sample_bool_cols = [col for col in df_cleaned.columns if col.startswith('Q06_')][:3]
for col in sample_bool_cols:
    if col in df_cleaned.columns:
        print(f"✓ {col}: dtype={df_cleaned[col].dtype}, unique values={df_cleaned[col].unique()}")


=== Verification ===

✓ Township column created successfully
  Total townships: 558 non-null values
  Unique townships: 117
  ✓ Township positioned correctly after Region (positions 8, 9)
  Sample townships: ['Aunglan', 'Kangyidaunt', 'Yamethin', 'Danubyu', 'Myothit', 'Pyapon', 'Aunglan', 'Myothit', 'Magway', 'Salin']

  Top 10 townships by count:
Township
Myothit           26
Mawlamyinegyun    21
Pwintbyu          19
Taungdwingyi      15
Aunglan           14
Pyawbwe           14
Pyay              12
Minbu             12
Thayarwady        12
Paungde           12
Name: count, dtype: int64

✓ Individual township columns successfully removed

✓ Q31 columns (13): ['Q31_a', 'Q31_b', 'Q31_c', 'Q31_d', 'Q31_e', 'Q31_f', 'Q31_g', 'Q31_h', 'Q31_i', 'Q31_j', 'Q31_k', 'Q31_l', 'Q31_other_text']
  Q31_j (None of the above) present: True

✓ Q33 columns (12): ['Q33_a', 'Q33_b', 'Q33_c', 'Q33_d', 'Q33_e', 'Q33_f', 'Q33_g', 'Q33_h', 'Q33_i', 'Q33_j', 'Q33_k', 'Q33_other_text']
  Q33_g (None of the ab

In [3]:
# Generate CLEAN_FY26_ANSWER.csv - mapping of column codes to questions and answers
import pandas as pd
import re

# Read the raw CSV to get original column names
df_raw = pd.read_csv('Digital_Digital_Access__Behavior_Survey_DAB_-_all_versions_-_English_eng_-_2025-12-29-02-53-07.csv', delimiter=';')

# Create answer mapping
answer_data = []

for col in df_raw.columns:
    # Special case: Q01.1 -> Q00
    if col.startswith('Q01.1'):
        answer_data.append({
            'code': 'Q00',
            'question': 'Do you have own phone?',
            'option': ''
        })
        continue
    
    # Sub-question pattern: Q##[_-]text/option
    if match := re.match(r'(Q\d+)[_-](.*[?\)"])/(.*)', col):
        q_code = match.group(1)
        question_text = match.group(2).strip()
        answer_text = match.group(3).strip()
        
        # Find which sub-code this would be (a, b, c, etc.)
        # We need to count how many times this Q code appeared before
        sub_index = len([d for d in answer_data if d['code'].startswith(q_code + '_')])
        sub_code = f"{q_code}_{chr(97 + sub_index)}"
        
        answer_data.append({
            'code': sub_code,
            'question': question_text,
            'option': answer_text
        })
    
    # Follow-up pattern: Q##[_-]If other...
    elif match := re.match(r'(Q\d+)[_-](If .*)', col):
        q_code = match.group(1)
        follow_up_text = match.group(2).strip()
        follow_up_code = f"{q_code}_other_text"
        
        # Skip duplicates (some columns appear multiple times in raw data)
        if not any(d['code'] == follow_up_code for d in answer_data):
            answer_data.append({
                'code': follow_up_code,
                'question': follow_up_text,
                'option': ''
            })
    
    # Main question pattern: Q##[_-]text
    elif match := re.match(r'(Q\d+)[_-](.*)', col):
        q_code = match.group(1)
        question_text = match.group(2).strip()
        
        # Skip if we already have this Q code (from sub-questions or duplicates)
        if not any(d['code'] == q_code for d in answer_data):
            answer_data.append({
                'code': q_code,
                'question': question_text,
                'option': ''
            })

# Create DataFrame and save
df_answer = pd.DataFrame(answer_data)

# Sort by code for better readability
df_answer = df_answer.sort_values('code').reset_index(drop=True)

# Save to CSV
df_answer.to_csv('CLEAN_FY26_ANSWER.csv', index=False)

print(f"Created CLEAN_FY26_ANSWER.csv with {len(df_answer)} column mappings")
print(f"\nSample mappings:")
print(df_answer.head(15).to_string(index=False))

Created CLEAN_FY26_ANSWER.csv with 241 column mappings

Sample mappings:
 code                                                                                                     question          option
  Q00                                                                                       Do you have own phone?                
  Q01                                                                                        Farmer's phone number                
  Q02 What is your age? (Enumerator instruction: Do NOT read options; classify response according to the options.)                
  Q03         What is your gender? (Enumerator instruction: READ all options. Make farmer choose one option only.)                
  Q06                                                                which crop did you grow during past 3 months?                
Q06_a                                                                which crop did you grow during past 3 months?            Rice
Q06_b     